<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-Jatto27/blob/main/Assignment_3_completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Joseph Atto

**Student Number**: 400381939

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [63]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Data-Analytics-with-Python/individual-assignment-i-Jatto27/e903f5dc36be27413311f5dcbd0ea45516f1a201/Data.csv")

#drop rows with missing values
missing_values = df.isnull()
df = df.dropna()

#drop columns that are constant
constant_columns = df.columns[df.nunique() == 1]
for i in constant_columns:
  df.drop(i, axis=1, inplace=True)

#Converting Dt_customer to number of days with the company
def days_since(date):
    return (pd.to_datetime('today') - pd.to_datetime(date)).days

#applying this function to each column
df['Dt_Customer'] = df['Dt_Customer'].apply(days_since)

#Convert Year_Birth to Customer age
df['Year_Birth'] = 2025 - df['Year_Birth']

#Remove Yolo, Absurd, and Alone Marital Status
df = df[df['Marital_Status'] != 'YOLO']
df = df[df['Marital_Status'] != 'Alone']
df = df[df['Marital_Status'] != 'Absurd']

#Display Head
display(df.head())

print(f"Number of records now: (rows): {df.shape[0]}")


/tmp/ipython-input-1656295046.py:15: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return (pd.to_datetime('today') - pd.to_datetime(date)).days


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,68,Graduation,Single,58138.0,0,0,4978,58,635,...,10,4,7,0,0,0,0,0,0,1
1,2174,71,Graduation,Single,46344.0,1,1,4132,38,11,...,1,2,5,0,0,0,0,0,0,0
2,4141,60,Graduation,Together,71613.0,0,0,4479,26,426,...,2,10,4,0,0,0,0,0,0,0
3,6182,41,Graduation,Together,26646.0,1,0,4072,26,11,...,0,4,6,0,0,0,0,0,0,0
4,5324,44,PhD,Married,58293.0,1,0,4328,94,173,...,3,6,5,0,0,0,0,0,0,0


Number of records now: (rows): 2209


# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [64]:
from sklearn.model_selection import train_test_split

#Split into 2 data frames (dropping ID column)
y = df['MntWines']
X = df.drop(columns=['MntWines', 'ID'])

#splitting into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [65]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

#OneHotEncodign to convert education and marital status into numerals
ct = ColumnTransformer(
    transformers = [
        ("cat", OneHotEncoder(sparse_output=False, drop="first"), ['Education', 'Marital_Status'])
    ],
    remainder = "passthrough"
)

#Building the pipeline with random forest
from sklearn.pipeline import Pipeline

pipe = Pipeline(
    steps= [
        ("preprocess", ct),
        ("rf_model", RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=3,
            random_state=20
        ))
    ]
)

pipe.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('rf_model',
                 RandomForestRegressor(max_depth=10, min_samples_leaf=3,
                                       min_samples_split=5, random_state=20))])

(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [66]:
#Create New data for the sample customer
sample = pd.DataFrame({
    'Year_Birth': [48],
    'Education': ['Graduation'],
    'Marital_Status': ['Married'],
    'Income': [80000],
    'Kidhome': [1],
    'Teenhome': [1],
    'Dt_Customer': ['2016-10-10'],
    'Recency': [43],
    'NumDealsPurchases': [2],
    'NumWebPurchases': [1],
    'NumCatalogPurchases': [0],
    'NumStorePurchases': [15],
    'NumWebVisitsMonth': [5],
    'AcceptedCmp1': [0],
    'AcceptedCmp2': [0],
    'AcceptedCmp3': [0],
    'AcceptedCmp4': [0],
    'AcceptedCmp5': [0],
    'Complain': [0],

    # adding missing columns:
    'Response': [0],
    'MntSweetProducts': [0],
    'MntMeatProducts': [0],
    'MntGoldProds': [0],
    'MntFruits': [0],
    'MntFishProducts': [0]
})

sample['Dt_Customer'] = sample['Dt_Customer'].apply(days_since)

print("Spending on wine: $", pipe.predict(sample).round(2)[0])
print("Output: ", pipe.predict(sample))

Spending on wine: $ 298.91
Output:  [298.90574235]


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [67]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import mean_absolute_error


y_pred = pipe.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)


#mean wine spending
print("Mean wine spending:", df['MntWines'].mean())

MAE: 87.98459707706098
RMSE: 151.68006078942523
R2: 0.815366441929353
Mean wine spending: 305.194205522861


I would choose R2 and and Mean Absolute Error (MAE) as two computational metrics though I also included a calculation for RMSE as well.

The MAE of 87.98 shows that on average, the model is off by $87.98. The Root mean square error (RMSE) is similar to MAE but the calculation squares the error, creating a larger RMSE when big mistakes occur.

Overall, the R2 of 0.815 explains how much variation in the wine spending is explained by my model. Overall, an R2 of 0.815 shows that the model fits the data quite well and I would say that it has acceptable moddeling performance.

(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [68]:
from sklearn.model_selection import cross_val_score
import numpy as np

# 6-fold cross validation using R2 as the scoring metric
cv_scores = cross_val_score(
    pipe,
    X, y,
    cv=6,
    scoring="r2"
)

print("R2 for each fold:", cv_scores)
print("Mean R2:", np.mean(cv_scores))
print("Std of R2:", np.std(cv_scores))

R2 for each fold: [0.83070099 0.81387749 0.77213696 0.82733228 0.77739377 0.82943805]
Mean R2: 0.8084799253139471
Std of R2: 0.024510962381951096


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [71]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "rf_model__n_estimators": [50, 100],      # number of trees
    "rf_model__max_depth": [5, 10, 15],       # max depth
    "rf_model__min_samples_split": [3, 6],    # min samples to split
    "rf_model__min_samples_leaf": [2, 4, 8],  # min samples in leaf
}

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="r2",   # same metric as to keep it consistent
    cv=6,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best hyperparameter combo:", grid_search.best_params_)
print("Best CV score (R2):", grid_search.best_score_)

# This is the best model
best_model = grid_search.best_estimator_

Best hyperparameter combo: {'rf_model__max_depth': 15, 'rf_model__min_samples_leaf': 2, 'rf_model__min_samples_split': 3, 'rf_model__n_estimators': 100}
Best CV score (R2): 0.7951479826093645


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [74]:
#New linear regression pipeline in place of random forest
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import mean_absolute_error

#Linear Regression
lin_pipe = Pipeline(steps=[
    ("prep", ct),
    ("linear", LinearRegression())
])

lin_pipe.fit(X_train, y_train)

#Evaluate the model
y_pred_lin = lin_pipe.predict(X_test)
mae_lin = mean_absolute_error(y_test, y_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
r2_lin = r2_score(y_test, y_pred_lin)

print("Linear Regression RMSE:", rmse_lin)
print("Linear Regression R2:", r2_lin)
print("Linear Regression MAE:", mae_lin)


#Compare with Best Random Forest Model
y_pred_rf = best_model.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("RF RMSE:", rmse_rf)
print("RF R2:", r2_rf)


Linear Regression RMSE: 175.3686106413765
Linear Regression R2: 0.7531930565786815
Linear Regression MAE: 117.96495667877669
RF RMSE: 151.66359561976648
RF R2: 0.8154065244272407


We can see through the better R2 value and the lower RMSE that the Random Forest Model is superior to the Linear Regression Model

(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [77]:
import plotly.express as px
import pandas as pd
#Creating Residuals
abs_res_lin = np.abs(y_test - y_pred_lin)
abs_res_rf = np.abs(y_test - y_pred_rf)

#Plotting the residuals
res_df = pd.DataFrame({
    "LR_residual": abs_res_lin,
    "RF_residual": abs_res_rf
})

fig = px.scatter(
    res_df,
    x="LR_residual",
    y="RF_residual",
    labels={"LR_residual": "Linear Reg absolute error", "RF_residual": "Random Forest absolute error"},
    title="Comparison of absolute prediction errors"
)

#Make the 45 degree reference line
min_val = 0
max_val = 800
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)

Visually, if a point is above the line, that means Random Forest had a greater absolute error and Linear Regression model is better for that point whereas if it is below the line, the Random Forest model would have a lower absolute error and better fit for the point.

It is difficult to tell, but visually, it appears that more points lie below the line indicating that Random Forest is the more accurate model. I confirmed this in the code below to see where most points lie. It confirmed that ~73% of test points are predicted better by the random forest model.

In [78]:
# Step 1: absolute residuals
abs_res_lin = np.abs(y_test - y_pred_lin)
abs_res_rf = np.abs(y_test - y_pred_rf)

# Step 2: Compare point-by-point
comparison = pd.DataFrame({
    "LR_abs_error": abs_res_lin,
    "RF_abs_error": abs_res_rf
})

# Who wins each test point?
comparison["winner"] = np.where(
    comparison["LR_abs_error"] < comparison["RF_abs_error"], "Linear Regression",
    np.where(
        comparison["RF_abs_error"] < comparison["LR_abs_error"], "Random Forest",
        "Tie"
    )
)

# Step 3: Count wins
counts = comparison["winner"].value_counts()

print(counts)
print("\nPercentage breakdown:")
print((counts / len(comparison) * 100).round(2))

winner
Random Forest        323
Linear Regression    119
Name: count, dtype: int64

Percentage breakdown:
winner
Random Forest        73.08
Linear Regression    26.92
Name: count, dtype: float64


# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [82]:
q1, q2 = df['MntMeatProducts'].quantile([1/3, 2/3])
print("33.33rd percentile:", q1)
print("66.67th percentile:", q2)

#Categorizing as low, medium, high
def label_meat_spending(x):
    if x < q1:
        return "low"
    elif x > q2:
        return "high"
    else:
        return "medium"

df["MeatSpendingLabel"] = df["MntMeatProducts"].apply(label_meat_spending) #applying the function
df.head()



33.33rd percentile: 24.0
66.67th percentile: 147.0


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,MeatSpendingLabel
0,5524,68,Graduation,Single,58138.0,0,0,4978,58,635,...,4,7,0,0,0,0,0,0,1,high
1,2174,71,Graduation,Single,46344.0,1,1,4132,38,11,...,2,5,0,0,0,0,0,0,0,low
2,4141,60,Graduation,Together,71613.0,0,0,4479,26,426,...,10,4,0,0,0,0,0,0,0,medium
3,6182,41,Graduation,Together,26646.0,1,0,4072,26,11,...,4,6,0,0,0,0,0,0,0,low
4,5324,44,PhD,Married,58293.0,1,0,4328,94,173,...,6,5,0,0,0,0,0,0,0,medium


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [87]:
X = df.drop(columns=["MeatSpendingLabel"])   # all features EXCEPT the label
y = df["MeatSpendingLabel"]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

cat_cols = ["Education", "Marital_Status"]
num_cols = [col for col in X.columns if col not in cat_cols]

ct = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(sparse_output=False, drop="first"), cat_cols),
        ("num", StandardScaler(), num_cols),
    ],
    remainder="drop"  # what to do with the rest ("drop" or "passthrough")
)

from sklearn.neighbors import KNeighborsClassifier

#Building the Pipeline
knn_pipe = Pipeline(steps=[
    ("prep", ct),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

knn_pipe.fit(X_train, y_train)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Education',
                                                   'Marital_Status']),
                                                 ('num', StandardScaler(),
                                                  ['ID', 'Year_Birth', 'Income',
                                                   'Kidhome', 'Teenhome',
                                                   'Dt_Customer', 'Recency',
                                                   'MntWines', 'MntFruits',
                                                   'MntMeatProducts',
                                                   'MntFishProducts',
                                                   'MntSweetProducts',
                                                   'MntGoldProds',
                                                   'NumDealsPurchases',
                                                   'NumWebPurchases',
                                                   'NumCatalogPurchases',
                                                   'NumStorePurchases',
                                                   'NumWebVisitsMonth',
                                                   'AcceptedCmp3',
                                                   'AcceptedCmp4',
                                                   'AcceptedCmp5',
                                                   'AcceptedCmp1',
                                                   'AcceptedCmp2', 'Complain',
                                                   'Response'])])),
                ('knn', KNeighborsClassifier())])

(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [88]:
from sklearn.metrics import accuracy_score, classification_report
y_pred_knn = knn_pipe.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn))

Accuracy: 0.8280542986425339

Classification Report:
               precision    recall  f1-score   support

        high       0.89      0.86      0.88       147
         low       0.83      0.91      0.87       146
      medium       0.76      0.71      0.74       149

    accuracy                           0.83       442
   macro avg       0.83      0.83      0.83       442
weighted avg       0.83      0.83      0.83       442



# Describe how you used Gen. AI. in this assignment (2 points)

I used generative AI to clarify machine learning concepts (cross-validation, pipelines, evaluation metrics) and to help structure my code for Random Forest, Linear Regression, and KNN models. The AI provided explanations and example snippets, but I implemented, tested, and verified all code and interpretations myself.


**Note**: The remaining 5 points will be assigned to readability of the work.